In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [2]:
!pip install ctransformers accelerate peft keybert jieba chinese-synonym-word

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 29.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 33.8 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 7.9 MB/s eta 0:00:00
  Created wheel for keybert: filename=keybert-0.8.4-py3-none-any.whl size=39199 sha256=f7e2db494bbd4b16db44de4f822f49502c4cd89329e26dc41ef8820f873ac2a9
  Stored in directory: /root/.cache/pip/wheels/97/ef/4c/6588bd7072b0cc04225b40e639b991e49ebd4e21fb81f0acee
Successfully built keybert


**Load Llama2 for RAG**

In [3]:
from ctransformers import AutoModelForCausalLM
from transformers import AutoTokenizer, pipeline

# Set gpu_layers to the number of layers to offload to GPU. Set to 0 if no GPU acceleration is available on your system.
# Model
model_llama = AutoModelForCausalLM.from_pretrained(
    "SinpxAI/Llama2-Chinese-7B-Chat-GGUF",
    model_file="llama2-chinese-7b-chat.Q4_K_M.gguf",
    model_type="llama",
    gpu_layers=110,
    hf=True
)

# Tokenizer
tokenizer_llama = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf", use_fast=True)

2024-04-20 07:55:40.290498: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-20 07:55:40.290606: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-20 07:55:40.630133: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

llama2-chinese-7b-chat.Q4_K_M.gguf:   0%|          | 0.00/4.08G [00:00<?, ?B/s]

ggml_cuda_set_main_device: using device 0 (Tesla T4) as main device


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [4]:
# Pipeline for Llama2
pipe_llama = pipeline(task="text-generation", model=model_llama, tokenizer=tokenizer_llama)

**Load Fine-tuned Bart Model & Tokenizer**

In [5]:
from transformers import BartForConditionalGeneration, AutoTokenizer, pipeline
from peft import PeftModel
import torch

base_model = "fnlp/bart-base-chinese"
new_model = "tonyma163/bart_v1"

device="cuda:0"

base_model_reload = BartForConditionalGeneration.from_pretrained(
        base_model,
        return_dict=True,
        low_cpu_mem_usage=True,
        torch_dtype=torch.float16,
        device_map=device,
        #trust_remote_code=True,
)
base_model_reload.half()

model_bart = PeftModel.from_pretrained(base_model_reload, new_model)

config.json:   0%|          | 0.00/1.69k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/561M [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/650 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/7.09M [00:00<?, ?B/s]

In [6]:
from transformers import BertTokenizer

tokenizer_bart = BertTokenizer.from_pretrained(base_model, trust_remote_code=True)

tokenizer_bart.pad_token = tokenizer_bart.eos_token
tokenizer_bart.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/479 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/259k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

In [7]:
from transformers import Text2TextGenerationPipeline

pipe_bart = Text2TextGenerationPipeline(model=model_bart, tokenizer=tokenizer_bart)

The model 'PeftModelForSeq2SeqLM' is not supported for . Supported models are ['BartForConditionalGeneration', 'BigBirdPegasusForConditionalGeneration', 'BlenderbotForConditionalGeneration', 'BlenderbotSmallForConditionalGeneration', 'EncoderDecoderModel', 'FSMTForConditionalGeneration', 'GPTSanJapaneseForConditionalGeneration', 'LEDForConditionalGeneration', 'LongT5ForConditionalGeneration', 'M2M100ForConditionalGeneration', 'MarianMTModel', 'MBartForConditionalGeneration', 'MT5ForConditionalGeneration', 'MvpForConditionalGeneration', 'NllbMoeForConditionalGeneration', 'PegasusForConditionalGeneration', 'PegasusXForConditionalGeneration', 'PLBartForConditionalGeneration', 'ProphetNetForConditionalGeneration', 'SeamlessM4TForTextToText', 'SeamlessM4Tv2ForTextToText', 'SwitchTransformersForConditionalGeneration', 'T5ForConditionalGeneration', 'UMT5ForConditionalGeneration', 'XLMProphetNetForConditionalGeneration'].


**Load Knowledge Set**

In [8]:
import pandas as pd
import ast

file_path = "/kaggle/input/nlp-knowledge-set/knowledge_set.txt"

data = []

# Open the file and parse each line from string to tuple
with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        if line.strip():  # Ensure the line is not empty
            try:
                # Convert string representation of tuple to actual tuple
                tuple_data = ast.literal_eval(line.strip())
                data.append(tuple_data)
            except SyntaxError:
                print(f"Skipping malformed line: {line.strip()}")

# Load the data into a DataFrame
df = pd.DataFrame(data, columns=['Entity', 'Category', 'Answer'])

In [9]:
df.head()

,Entity,Category,Answer
0,西宁,2018-11-14,"阴,东风,最高气温:5℃,最低气温:-4℃"
1,何霄玲,喜好,poi
2,快乐大本营之快乐到家,评论,不好意思啊坡姐，我是你的路人黑，对不住了
3,辣相见川菜（三水总店）,特色菜,水煮鱼
4,浮城大亨,评论,人生是一幕大剧


**Clean Knowledge Set**

In [61]:
distinct_categories = df['Category'].unique()
distinct_categories

array(['2018-11-14', '喜好', '评论', '特色菜', '简介', '适合听', '评分', '人均价格', '喜欢',
       '适合吃', '2018-10-13', '主演', '2018-10-16', '演唱', '2018-10-15',
       '2018-12-25', '2018-12-27', '类型', '获奖', '成就', '嘿玛口碑', '(live)评论',
       '身高', '2018-11-12', '无持续风向,最高气温:12℃,最低气温:-4℃适合吃', '2018-12-24',
       '2018-11-19', '订单量', '导演', '2018-11-2', '喜欢的新闻',
       '东风,最高气温:13℃,最低气温:6℃适合吃', '东北风,最高气温:6℃,最低气温:0℃适合吃',
       '西南风,最高气温:4℃,最低气温:-5℃适合吃', '人间道评论', '2018-10-26', '新闻', '口碑',
       '东南风,最高气温:5℃,最低气温:-3℃适合吃', '星座', '2018-11-11', '2018-12-21',
       '2018-10-11', '国家地区', '2018-12-12', '2018-10-23', '2018-11-8',
       '东南风,最高气温:11℃,最低气温:1℃适合吃', '2018-10-1', '情绪零碎评论', '成分', '我的姓氏评论',
       '3040评论', '地址', '2018-12-26', '嘿玛导演', '无持续风向,最高气温:19℃,最低气温:4℃适合吃',
       '东南风,最高气温:16℃,最低气温:11℃适合吃', '西北风,最高气温:8℃,最低气温:0℃适合吃', '2018-10-12',
       '2018-12-17', '时间', '2018-10-4', '道道道评论', '(00Live)(live版)评论',
       '无持续风向,最高气温:15℃,最低气温:11℃适合吃', '2018-11-1',
       '无持续风向,最高气温:13℃,最低气温:5℃适合吃', '2018-12-16', '

In [10]:
def categoryClean(df , targetCategory):
        
        targetCategoryLen = len(targetCategory)
        #print("\n Target Category Length " + targetCategory + " :" , targetCategoryLen)
        #print("------------------------------------------------------------------------------------------------------------------------")
        
        # Count the "Category" value containing "targetCategory"
        comment_count = df[df['Category']== targetCategory].shape[0]
        #print("\nNumber of rows where 'Category' contains " + targetCategory + " :", comment_count)
        
        #print("------------------------------------------------------------------------------------------------------------------------")
        filtered_data = df[df['Category'].str.contains(targetCategory) & (df['Category'].str.len() > targetCategoryLen)]
        #print(filtered_data)
        
        #print("------------------------------------------------------------------------------------------------------------------------")
        targetRow_ids = df[df['Category'].str.contains(targetCategory) & (df['Category'].str.len() > targetCategoryLen)].index
        #print(targetRow_ids)
        
        #print("------------------------------------------------------------------------------------------------------------------------")
        distinct_categories = df[df['Category'].str.contains(targetCategory)]['Category'].unique()
        #print("Distinct categories containing " + targetCategory + " :", distinct_categories)
        
        ## update 
        df.loc[df['Category'].str.contains(targetCategory), 'Entity'] = df['Entity'] + df['Category'].str.split(targetCategory).str[0]
        df['Category'] = df['Category'].apply(lambda x: targetCategory if targetCategory in x else x)
        
        # Count the "Category" value containing "targetCategory"
        comment_count = df[df['Category'] == targetCategory].shape[0]
        #print("\nNumber of rows where 'Category' contains " + targetCategory + " :", comment_count)
        
        #print("------------------------------------------------------------------------------------------------------------------------")
        filtered_data = df[df['Category'].str.contains(targetCategory) & (df['Category'].str.len() > targetCategoryLen)]
        #print(filtered_data)
        
        #print("------------------------------------------------------------------------------------------------------------------------")
        distinct_categories = df[df['Category'].str.contains(targetCategory)]['Category'].unique()
        #print("Distinct categories containing " + targetCategory + " :", distinct_categories)
        
        return df

In [11]:
df['Entity'][326]

'倩女幽魂II：'

In [12]:
cateList = ['评论', '主演','口碑','导演','类型','评分','国家地区','适合吃','特色菜','人均价格', '订单量','地址',]

for cat in cateList:
    df = categoryClean(df, cat)

In [13]:
distinct_categories = df['Category'].unique()
distinct_categories

array(['2018-11-14', '喜好', '评论', '特色菜', '简介', '适合听', '评分', '人均价格', '喜欢',
       '适合吃', '2018-10-13', '主演', '2018-10-16', '演唱', '2018-10-15',
       '2018-12-25', '2018-12-27', '类型', '获奖', '成就', '口碑', '身高',
       '2018-11-12', '2018-12-24', '2018-11-19', '订单量', '导演', '2018-11-2',
       '喜欢的新闻', '2018-10-26', '新闻', '星座', '2018-11-11', '2018-12-21',
       '2018-10-11', '国家地区', '2018-12-12', '2018-10-23', '2018-11-8',
       '2018-10-1', '成分', '地址', '2018-12-26', '2018-10-12', '2018-12-17',
       '时间', '2018-10-4', '2018-11-1', '2018-12-16', '2018-12-7',
       '2018-12-3', '2018-12-13', '血型', '2018-12-6', '2018-11-25',
       '2018-11-26', '2018-12-14', '2018-12-9', '2018-12-19', '2018-10-8',
       '2018-10-27', '2018-10-24', '2018-12-8', '2018-12-2', '2018-11-5',
       '2018-10-19', '体重', '2018-12-15', '2018-10-25', '2018-10-21',
       '2018-10-5', '2018-11-18', '2018-10-20', '2018-10-18',
       '2018-12-23', '2018-10-3', '2018-10-9', '2018-10-6', '2018-11-23',
       '2018-11-3'

In [14]:
df['Entity'][326]

'倩女幽魂II：人间道'

**Knowledge Tree**

In [15]:
import networkx as nx
import pandas as pd

# Assuming 'df' is your DataFrame containing the data
G = nx.DiGraph()  # Directed graph can still function as a tree

# Add nodes and edges based on the DataFrame
for index, row in df.iterrows():
    entity_id = f"Entity: {row['Entity']}"
    category_id = f"Category: {row['Category']} ({row['Entity']})"
    answer_id = f"Answer: {row['Answer']} ({row['Category']})"

    # Ensure that nodes for each level (entity, category) are unique per entity-category pair
    if entity_id not in G:
        G.add_node(entity_id, type='Entity', name=row['Entity'])
    if category_id not in G:
        G.add_node(category_id, type='Category', name=row['Category'])
    
    # Answers can be multiple per category, so they are always added
    G.add_node(answer_id, type='Answer', content=row['Answer'])
    
    # Connect nodes hierarchically
    G.add_edge(entity_id, category_id)
    G.add_edge(category_id, answer_id)

**Part of Speech Function**

In [16]:
def checkSentence(sentence):
    separated_sentences = []
    temp_sentence = ""
    in_bracket = False

    for char in sentence:
        if char == "『" or char == "《":
            in_bracket = True
            if temp_sentence:
                separated_sentences.append(temp_sentence)
                temp_sentence = ""
            temp_sentence += char
        elif char == "』" or char == "》":
            in_bracket = False
            temp_sentence += char
            separated_sentences.append(temp_sentence)
            temp_sentence = ""
        elif char == "。" and not in_bracket:
            separated_sentences.append(temp_sentence)
            temp_sentence = ""
        else:
            temp_sentence += char

    if temp_sentence:
        separated_sentences.append(temp_sentence)

    return separated_sentences

In [17]:
import jieba.posseg as pseg

def extractPOS(sentences):

    result = []
    special_words = []

    for sentence in sentences:
        if '『' in sentence or '《' in sentence:
            # Remove the brackets
            sentence = sentence.replace('『', '@').replace('』', '@').replace('《', '@').replace('》', '@')
            #result.append(sentence)
            special_words.append(sentence)
        else:
            allowSentencePOS = ("a", "ad", "ag", "an", ## 形容词 
                                "b", # 区别词
                                "c", # 连词
                                "f", 
                                "g", 
                                "h",
                                "i", 
                                "j", 
                                "k", 
                                "l", 
                                "m", "mg", "mq", 
                                "n", "ng", "nr", "nrfg", "nrt","ns", "nt", "nz", 
                                "o", 
                                "q", 
                                "s", 
                                "t", "tg",
                                "v", "vd", "vg", "vi", "vn","vq" ## 动词
                                )

            exclude = ( "p", ## 介词
                "d", "df", "dg", # 副词
                "e", # 叹词 
                "r", "rg", "rr", "rz",  ## 代词
                "u",  "ud", "ug", "uj", "ul", "uv", "uz",  ## 助词
                "x",  ## 非语素词（包含标点符号
                "y", ## 语气词
                "z","zg" ## 助词
              )

            sentence = pseg.lcut(sentence)
            seg_words = [word for word, pos in sentence if pos in allowSentencePOS]
            result.extend(seg_words)
            
    return special_words+result

**Retrieve Data from Knowledge Tree**

In [18]:
def has_common_character(keyword, category):
    return any(char in category for char in keyword)

In [149]:
import jieba
import random

def retrieve_answers(graph, query):
    segmented_keywords = extractPOS(checkSentence(query))
    special_keywords = [kw.strip('@') for kw in segmented_keywords if kw.startswith('@')]
    regular_keywords = [kw for kw in segmented_keywords if not kw.startswith('@')]

    category_answers = {}  # Store answers grouped by categories
    special_categories = {'新闻', '评论'}

    # Top-Down Search: From entities to categories to answers
    for entity_node in (n for n in graph.nodes if graph.nodes[n]['type'] == 'Entity'):
        entity_keywords = [kw for kw in regular_keywords if kw == graph.nodes[entity_node]['name']]
        if entity_keywords:
            for category_node in graph.successors(entity_node):
                category_keywords = [kw for kw in regular_keywords if has_common_character(kw, graph.nodes[category_node]['name'])]
                if category_keywords:
                    for answer_node in graph.successors(category_node):
                        category_name = graph.nodes[category_node]['name']
                        answer_content = graph.nodes[answer_node]['content']
                        if category_name not in category_answers:
                            category_answers[category_name] = []
                        if category_name not in special_categories:
                            category_answers[category_name].append(category_name+answer_content)
                        else:
                            category_answers[category_name].append(answer_content)

    # Bottom-Up Search: From answers to categories to entities
    for answer_node in (n for n in graph.nodes if graph.nodes[n].get('type') == 'Answer'):
        if any(kw == graph.nodes[answer_node]['content'] for kw in special_keywords):
            for category_node in graph.predecessors(answer_node):
                category_name = graph.nodes[category_node]['name']
                if any(has_common_character(kw, category_name) for kw in regular_keywords):
                    for entity_node in graph.predecessors(category_node):
                        if category_name not in category_answers:
                            category_answers[category_name] = []
                        category_answers[category_name].append(graph.nodes[entity_node]['name'] + category_name)

    final_answers = []
    require_llama = False
    for category, answers in category_answers.items():
        if category in special_categories:
            # Randomly pick one answer if category is special
            final_answers.append(random.choice(answers))
        else:
            # Collect all answers for other categories
            final_answers.extend(answers)
            require_llama = True  # Indicate further processing may be needed for non-special categories

    return final_answers, require_llama

In [150]:
#Q1
input_query = "你知道张国荣的星座吗？"
answers = retrieve_answers(G, input_query)
print(f"Question: {input_query} Response: {answers}")

#Q2
input_query = "周迅的星座是什么?"
answers = retrieve_answers(G, input_query)
print(f"Question: {input_query} Response: {answers}")

#Q3
input_query = "范冰冰多重啊？"
answers = retrieve_answers(G, input_query)
print(f"Question: {input_query} Response: {answers}")

#Q4
input_query = "『爱你等于爱自己』这首歌的主唱是谁啊？"
answers = retrieve_answers(G, input_query)
print(f"Question: {input_query} Response: {answers}")

#Q5
input_query = "电影《倾城》主演是谁?"
answers = retrieve_answers(G, input_query)
print(f"Question: {input_query} Response: {answers}")

#Q6
input_query = "说说张学友的新闻"
answers = retrieve_answers(G, input_query)
print(f"Question: {input_query} Response: {answers}")

#Q7
input_query = "我想知道网友对范冰冰的评论如何。"
answers = retrieve_answers(G, input_query)
print(f"Question: {input_query} Response: {answers}")

Question: 你知道张国荣的星座吗？ Response: (['星座处女座'], True)
Question: 周迅的星座是什么? Response: (['星座天秤座'], True)
Question: 范冰冰多重啊？ Response: (['体重56kg'], True)
Question: 『爱你等于爱自己』这首歌的主唱是谁啊？ Response: (['王力宏演唱'], True)
Question: 电影《倾城》主演是谁? Response: (['林心如主演'], True)
Question: 说说张学友的新闻 Response: (['9月30日，陕西咸阳"逃犯克星"张学友举行演唱会时再次"立功"咸阳公安通报，不仅有5名逃犯落网，还抓获了其它违法犯罪人员13人9月28日晚，张学友演唱会在石家庄举行时便成功抓获了3名逃犯向娱乐圈卧底探员张Sir致敬！“逃犯克星”张学友又立功5名逃犯在咸阳落网'], False)
Question: 我想知道网友对范冰冰的评论如何。 Response: (['除了“王者之气”我想不出什么词来形容她了'], False)


In [151]:

input_query = "你知道张柏芝的生日是什么时候吗？"
answers = retrieve_answers(G, input_query)
print(f"Question: {input_query} Response: {answers}")

Question: 你知道张柏芝的生日是什么时候吗？ Response: (['生日1980-5-24', '出生地中国香港'], True)


**Llama2(Knowledge Tree) + Fine-tuned Bart**

In [21]:
system_prompt = """
Please answer the following question based on the provided context.
Provide only the direct answers without any additional explanations or context.
Please only output answers only.
Please ensure that your responses are socially unbiased and positive in nature.
If a question does not make any sense or is not factually coherent, explain why instead of answering something not correct.
If you don't know the answer, please don't share false information.
"""

In [22]:
def query_llama(query, context):
    # Test
    prompt = f"""
    <s>[INST] <<SYS>>
    {system_prompt}
     <</SYS>> [/INST]</s>

    <s>[INST]
    Question: {query}
    Context: {context}
    Answer:
    [/INST]
        """

    output = pipe_llama(
        prompt,
        do_sample=True,
        max_new_tokens=256,
        top_k=40,
        top_p=0.95,
        temperature=0.75,
        return_full_text=False,
        repetition_penalty=1
    )
    return output[0]['generated_text']

In [23]:
def query_system(graph, query):
    # Attempt to retrieve answers from the knowledge graph
    answers, require_llama = retrieve_answers(graph, query)
    
    # If answers are found in the graph, return
    if answers and require_llama:
        # pass to llama
        return "Answer from knowledge graph:", query_llama(query, answers)
    elif answers and not require_llama:
        # return directly
        return "Answer from knowledge graph:", answers
    # If no answers are found, -> language model
    else:
        generated_answer = pipe_bart(query)
        return "Answer from language model:", generated_answer[0]['generated_text']

**Test**

In [152]:
query = "周迅的星座是什么？"
result = query_system(G, query)
print(result)

('Answer from knowledge graph:', '周迅的星座是天秤座。\n')


In [153]:
query = "你知道张柏芝的生日是什么时候吗？"
result = query_system(G, query)
print(result)

('Answer from knowledge graph:', '生日1980-5-24\n')


In [154]:
query = "『爱你等于爱自己』这首歌的主唱是谁啊？"
result = query_system(G, query)
print(result)

('Answer from knowledge graph:', '爱你等于爱自己\n主唱：王力宏\n')


In [155]:
query = "电影《倾城》主演是谁?"
result = query_system(G, query)
print(result)

('Answer from knowledge graph:', '林心如主演电影《倾城》。')


In [156]:
query = "帮我点一首《想你》吧"
result = query_system(G, query)
print(result)

('Answer from language model:', '正 在 为 你 播 放 《 想 你 》')


In [157]:
query = "张国荣哪里出生的？"
result = query_system(G, query)
print(result)

('Answer from knowledge graph:', '张国荣出生于香港九龙。\n')


In [158]:
query = "今天是什么日子了？"
result = query_system(G, query)
print(result)

('Answer from language model:', '今 天 是 2018 年 10 月 18 日')


In [159]:
query = "说说张学友的新闻"
result = query_system(G, query)
print(result)

('Answer from knowledge graph:', ['9月30日，陕西咸阳"逃犯克星"张学友举行演唱会时再次"立功"咸阳公安通报，不仅有5名逃犯落网，还抓获了其它违法犯罪人员13人9月28日晚，张学友演唱会在石家庄举行时便成功抓获了3名逃犯向娱乐圈卧底探员张Sir致敬！“逃犯克星”张学友又立功5名逃犯在咸阳落网'])


In [160]:
query = "说说张学友的新闻"
result = query_system(G, query)
print(result)

('Answer from knowledge graph:', ['据平安洛阳7月8日，民警在张学友演唱会现场附近执勤时发现一车辆交通违法，驾车男子提供个人信息不实，身份可疑。原来该男子从563公里远的武汉赶到洛阳，冒用他人信息无证驾驶，就是为看张学友演唱会。警察蜀黍棒棒哒(人民网)逃犯克星张学友上演第七杀？洛阳演唱会前一男子被抓'])


In [161]:
query = "我想知道网友对范冰冰的评论如何。"
result = query_system(G, query)
print(result)

('Answer from knowledge graph:', ['她骨子里其实是个爷们。'])
